## Example setup

In [3]:
library(shifttx)

shifttx: Estimate Causal Effects with Stochastic Treatments
Version: 0.0.0.9000


In [4]:
# Example based on the data-generating mechanism presented in the simulation
n <- 100
W <- data.frame(W1 = runif(n), W2 = rbinom(n, 1, 0.7))
A <- rpois(n, lambda = exp(3 + .3 * log(W$W1) - 0.2 * exp(W$W1) * W$W2))
Y <- rbinom(n, 1, plogis(-1 + 0.05 * A - 0.02 * A * W$W2 + 0.2 * A * tan(W$W1^2)
                         - 0.02 * W$W1 * W$W2 + 0.1 * A * W$W1 * W$W2))
fitA.0 <- glm(A ~ I(log(W1)) + I(exp(W1)):W2, family = poisson,
              data = data.frame(A, W))
fitY.0 <- glm(Y ~ A + A:W2 + A:I(tan(W1^2)) + W1:W2 + A:W1:W2,
              family = binomial, data = data.frame(A, W))

gn.0  <- function(A = A, W = W) {
  dpois(A, lambda = predict(fitA.0, newdata = W, type = "response"))
}

Qn.0 <- function(A = A, W = W) {
  predict(fitY.0, newdata = data.frame(A, W, row.names = NULL),
          type = "response")
}

## Original implementation (vdL & Diáz, 2012)

In [5]:
tmle_shift_new <- tmle_shift(Y = Y, A = A, W = W, Qn = Qn.0, gn = gn.0,
                             delta = 2, tol = 1e-4, iter_max = 5,
                             A_val = seq(1, 60, 1))

In [18]:
tmle_shift_new[1:2]

psi.hat    var.hat 
0.61510431 0.00307895

## 1-TMLE implementation (vdL & Diáz, 2017)

In [24]:
est_eqn2 <- function(eps, QnAW, Qn, H1, gn0d, EQnd, D2, prev_sum, Y, A, W,
                     delta) {

  sum((Y - (QnAW + eps * H1)) * H1 + (Qn(A + delta, W) - EQnd) -
      rowSums(D2 * exp(eps * D2 + prev_sum) * gn0d) /
      rowSums(exp(eps * D2 + prev_sum) * gn0d))
}

In [31]:
f_iter2 <- function(Qn, gn, gn0d = NULL, prev_sum = 0, first = FALSE, h_int,
                    Y, A, W, delta, A_val) {

  # numerical integrals and equation (7)
  Qnd <- t(sapply(seq_len(nrow(W)), function(i) Qn(A_val + delta, W[i, ])))
  gnd <- t(sapply(seq_len(nrow(W)), function(i) gn(A_val, W[i, ])))
  gnd <- gnd / rowSums(gnd)
  if (first) gn0d <- gnd

  EQnd <- rowSums(Qnd * gnd) * h_int
  D2 <- Qnd - EQnd
  QnAW <- Qn(A, W)
  H1 <- gn(A - delta, W) / gn(A, W)

  # equation (8)
  est_eqn_min  <- stats::uniroot(est_eqn2, c(-1, 1),  Y = Y, A = A, W = W,
                                 delta = delta, QnAW = QnAW, Qn = Qn, H1 = H1,
                                 gn0d = gn0d, EQnd = EQnd, D2 = D2,
                                 prev_sum = prev_sum)
  eps <- est_eqn_min$root

  # updated values
  gn_new   <- function(a, w) exp(eps * Qn(a + delta, w)) * gn(a, w)
  Qn_new   <- function(a, w) Qn(a, w) + eps * gn(a - delta, w) / gn(a, w)
  prev_sum <- prev_sum + eps * D2
  return(list(Qn = Qn_new, gn = gn,
              prev_sum = prev_sum, eps = eps, gn0d = gn0d))
}

In [36]:
tmle1_shift <- function(Y, A, W,
                        Qn, gn,
                        delta, A_val) {

  # interval partition length, A_val assumed equally spaced
  n_A_val <- length(A_val)
  h_int <- (A_val[n_A_val] - A_val[1]) / (n_A_val - 1)

  # inputs are initial estimators of Q and g; returns their updated values
  ini_out <- f_iter2(Qn = Qn, gn = gn, gn0d = NULL,
                     prev_sum = 0, first = TRUE, h_int = h_int,
                     W = W, A = A, A_val = A_val, Y = Y, delta = delta)
  gn0d <- ini_out$gn0d

  # update step: 1-TMLE requires we only do this for Qn
  new_out <- f_iter2(Qn = ini_out$Qn, gn = ini_out$gn, gn0d = gn0d,
                     prev_sum = ini_out$prev_sum, first = FALSE, h_int = h_int,
                     W = W, A = A, A_val = A_val, Y = Y, delta = delta)
  Qnd <- t(sapply(seq_len(nrow(W)), function(i) new_out$Qn(A_val + delta, W[i, ])))
  #gnd <- t(sapply(seq_len(nrow(W)), function(i) new_out$gn(A_val, W[i, ])))
  #gnd <- gnd / rowSums(gnd)

  # plug in tmle
  psi_hat <- mean(rowSums(Qnd * gn0d) * h_int)

  # influence curve of tmle
  IC <- (Y - new_out$Qn(A, W)) * new_out$gn(A - delta, W) / new_out$gn(A, W) +
    new_out$Qn(A + delta, W) - psi_hat
  var_hat <- stats::var(IC) / length(Y)
      
  # construct output object in S3 style
  out <- list(psi_hat = psi_hat, var_hat = var_hat, ic = as.vector(IC))
  class(out) <- "shift_1tmle"
  return(out)
  #return(c(psi.hat = psi_hat, var.hat = var_hat, IC = IC))
}

In [37]:
tmle1_new <- tmle1_shift(Y = Y, A = A, W = W,
                         Qn = Qn.0, gn = gn.0,
                         delta = 2, A_val = seq(1, 60, 1))

In [39]:
tmle1_new$psi
tmle1_new$var

[1] 0.6151143

[1] 0.003078206

## Check 1-TMLE is similar to iterative procedure

What? Are you serious? That was all...

In [40]:
all.equal(tmle1_new$psi_hat, tmle_shift_new[1])

[1] "names for current but not for target"  
[2] "Mean relative difference: 1.629422e-05"

In [41]:
all.equal(tmle1_new$var_hat, tmle_shift_new[2])

[1] "names for current but not for target"  
[2] "Mean relative difference: 0.0002415953"

Maybe the influence curve estimates are different?

In [42]:
all.equal(tmle1_new$ic, tmle_shift_new[3:102])

[1] "names for current but not for target"  
[2] "Mean relative difference: 9.895182e-05"